# Generic Box
This code communicates with a component. The component was launched with the command below from the root of the repository:

```shell
$ docker run --rm -it -p 4061:8061 -v ./src/external1.py=/workspace/external.py generic
```


In [1]:
import argparse
import grpc
import io
import matplotlib.pyplot as plt
#In case of importing files from other directory
from importlib.machinery import SourceFileLoader

vggt_pb2 = SourceFileLoader("vggt_pb2","../protos/vggt_pb2.py").load_module()
vggt_pb2_grpc = SourceFileLoader("vggt_pb2_grpc","../protos/vggt_pb2_grpc.py").load_module()

import PIL.Image as PIL_image
import numpy as np
import cv2
from scipy.io import loadmat,savemat

## Set parameters (GRPC) and data
Set IP and port of the service (the same used in docker run), input data (image in a matfile) and call the service

In [4]:
from PIL import Image
import json

#target='printart.isr.ist.utl.pt:8061'
target='localhost:8061'
# List of file paths (example paths — replace with your actual ones)
file_paths = [
    'images/00.jpg',
    'images/01.jpg',
]

#list all files in a certain directory
import os
directory = "/home/manuelf/mast3r/images_in/piv"
file_paths = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))][0:1]

# List to hold the binary data of each image
image_byte_list = []
for path in file_paths:
    # Open the file in binary read mode ('rb') and read its entire content
    with open(path, 'rb') as f:
        image_bytes = f.read()
        image_byte_list.append(image_bytes)
        print(f"Read {path}: {len(image_bytes) / (1024 * 1024):.2f} MB")

config_json = {
    "aispgradio": {
        "command": "3d_infer",
        "parameters": {
            "device": "cuda:0", # TODO: implement this
            "conf_vis": 50.0
        }
    }
}

request = vggt_pb2.VGGTRequest(images=image_byte_list, config_json = json.dumps(config_json))

#Para imagens muito grandes buffer grande
channel_opt = [('grpc.max_send_message_length', 512 * 1024 * 1024), ('grpc.max_receive_message_length', 512 * 1024 * 1024)]
channel=grpc.insecure_channel(target,options=channel_opt)
estimator_stub = vggt_pb2_grpc.VGGTServiceStub(channel)

Read /home/manuelf/mast3r/images_in/piv/6.png: 0.17 MB


In [5]:
#Name of the method in the service
response = estimator_stub.Forward(request)
#After finishing channel.close

In [10]:
import torch

def bytes_to_tensor(b: bytes) -> torch.Tensor:
    return torch.load(io.BytesIO(b))

depthmap = bytes_to_tensor(response.depth)

#plt.imshow(depthmap.squeeze()[0], cmap='gray')
bytes_to_tensor(response.world_points).shape


torch.Size([1, 1, 518, 518, 3])

In [13]:
import numpy as np
import pythreejs as three
import ipywidgets as widgets
from ipywidgets.embed import embed_minimal_html

# 1. Generate random data for the points

# Create random (x, y, z) coordinates between -5 and 5
positions = bytes_to_tensor(response.world_points).squeeze().numpy().reshape(-1,3)[::2]
colors = bytes_to_tensor(response.images).permute(0,2,3,1).numpy().reshape(-1,3)[::2] # Normalize to [0, 1]

# 2. Create the geometry and assign the point positions
# The data needs to be a float32 array
point_geometry_colored = three.BufferGeometry(
    attributes={
        'position': three.BufferAttribute(positions.astype('float32')),
        # === KEY CHANGE 2: Add the 'color' attribute to the geometry ===
        'color': three.BufferAttribute(colors.astype('float32'))
    }
)

# 3. Create a material for the points
point_material_colored = three.PointsMaterial(
    size=0.01, 
    # === KEY CHANGE 3: Tell the material to use the geometry's colors ===
    vertexColors='VertexColors'
)

# 4. Create the Points object
points_colored = three.Points(geometry=point_geometry_colored,
                               material=point_material_colored)

# 5. Set up the scene, camera, and renderer
camera_colored = three.PerspectiveCamera(
    position=[0, 0, 12],
    fov=50,
    aspect=1.5,
)
camera_colored.up = [0, -1, 0]

# Add an axes helper to see the orientation
axes_colored = three.AxesHelper(size=5)

scene_colored = three.Scene(children=[
    points_colored, 
    axes_colored,
    three.AmbientLight(color='#cccccc')
])

renderer_colored = three.Renderer(
    camera=camera_colored, 
    scene=scene_colored, 
    controls=[three.OrbitControls(controlling=camera_colored)],
    width=1200, 
    height=900
)

# Display the renderer in the notebook
renderer_colored

# Save as standalone HTML file
widget = widgets.VBox([renderer_colored])
with io.StringIO() as f:
    embed_minimal_html(f, views=[widget], title="3D Point Cloud")
    f.seek(0)
    file_like_object = f.read()
